<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/autoenc_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==2.1.0-rc0

     |████████████████████████████████| 402.3MB 36kB/s 
     |████████████████████████████████| 450kB 46.8MB/s 
     |████████████████████████████████| 3.8MB 52.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=ca08bdf6d9299565633b0c65797af5a7fe57f830b47bf512f982dc0579f21e0a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installatio

In [2]:
pip install keras==2.3.1

In [3]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 108MB/s]
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 105MB/s]
 84% 49.0M/58.3M [00:02<00:00, 16.0MB/s]
100% 58.3M/58.3M [00:02<00:00, 27.6MB/s]
 79% 41.0M/52.2M [00:01<00:00, 28.4MB/s]
100% 52.2M/52.2M [00:01<00:00, 43.2MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 76.0MB/s]


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
trn=pd.read_csv('train_transaction.csv.zip')
tst=pd.read_csv('test_transaction.csv.zip')
trn=trn.drop(['isFraud'],1)

In [6]:
ls=list(trn.filter(regex='V'))
trn=trn.drop(ls,1)
tst=tst.drop(ls,1)
trn.shape

(590540, 54)

In [0]:

def ext(fnl):

  fnl['P_1']=fnl['P_emaildomain'].fillna('nan').str.split('.').str[0]
  fnl['P_2']=fnl['P_emaildomain'].fillna('nan').str.split('.').str[1]


  fnl['R_1']=fnl['R_emaildomain'].fillna('nan').str.split('.').str[0]
  fnl['R_2']=fnl['R_emaildomain'].fillna('nan').str.split('.').str[1]
  return fnl
trn=ext(trn)
tst=ext(tst)

In [0]:

trn=trn.drop(['P_emaildomain','R_emaildomain','TransactionDT','TransactionID'],1)
tst=tst.drop(['P_emaildomain','R_emaildomain','TransactionDT','TransactionID'],1)

In [0]:
def handle_disc(df,cols,num):
  for col in cols:
    par=min(df[col].nunique(),num)
    unq = df[col].value_counts().head(par)
    dum=pd.get_dummies(df[col])[unq.index]
    ls=[]
    for i in dum.columns:
      ls.append(str(col)+str(i))
    dum.columns=ls
    df=pd.concat([df,dum],1)

  df=df.drop(cols,1)
  sp=df.shape[1]
  df.columns=list(range(sp))
  return df
def handle_cont(df,cols):
  for col in cols:
    df[col]=df[col].fillna(df[col].mean())
    mn=df[col].mean()
    std=df[col].std()
    df[col]=(df[col]-mn)/(std+0.1)
    df[col]=np.log10(df[col]+1-min(0,min(df[col])))
  return df[cols]
disc=['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2','P_1','P_2','R_1','R_2','M1','M2','M3','M4','M5','M6','M7','M8','M9']
cont=[i for i in tst.columns if i not in disc]
trn_cont=handle_cont(trn,cont)
tst_cont=handle_cont(tst,cont)
trn_cont=trn_cont.replace([np.inf,-np.inf],0)
tst_cont=tst_cont.replace([np.inf,-np.inf],0)
trn_disc=handle_disc(trn,disc,140)
tst_disc=handle_disc(tst,disc,140)

In [0]:

trn=pd.concat([trn_cont,trn_disc],1).reset_index(drop=True)
tst=pd.concat([tst_cont,tst_disc],1).reset_index(drop=True)

In [0]:
ls=list(set(list(tst)).intersection(set(list(trn))))
trn=trn[ls]
tst=tst[ls]

In [0]:

trn=trn.loc[:,~(trn.columns.duplicated())]
tst=tst.loc[:,~(tst.columns.duplicated())]

In [13]:
fnl=pd.concat([trn,tst],0)
fnl=fnl.reset_index(drop=True)
z=fnl.isna().sum()
z[z>0]

Series([], dtype: int64)

In [14]:
import gc
import os
gc.collect()
import numpy as np
import keras 
from math import ceil
from keras.models import Model
import matplotlib.pyplot as plt
from keras.utils import Sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Concatenate, Dropout,BatchNormalization
from keras.layers import Add
import keras.backend as K
class DAESequence(Sequence):
    def __init__(self,df,batch_size):
        self.batch_size=batch_size
        self.len_data=df.shape[0]
        self.columns=df.shape[1]
        self.data=df.values
        self.idx=[]
    def __len__(self):
        return int(ceil(self.len_data/self.batch_size))
    def __getitem__(self,idx):
        self.idx.append(idx)
        last=min((idx+1)*self.batch_size,self.len_data)
        idx=idx*self.batch_size
        size=last-idx
        output_x=self.data[idx:last]
        length=last-idx
        rnd=np.random.randint(0,self.len_data-size,size)
        rnd[rnd>idx]+=size
        cols=np.random.randint(0,self.columns,int(self.columns*0.1))
        noise_x=output_x.copy()
        noise_x[:,cols]=self.data[rnd[:,None],cols]
        return noise_x,output_x
from keras import backend as K


class WarmUpLearningRateScheduler(keras.callbacks.Callback):
    """Warmup learning rate scheduler
    """

    def __init__(self, warmup_batches, init_lr, verbose=0):
        """Constructor for warmup learning rate scheduler

        Arguments:
            warmup_batches {int} -- Number of batch for warmup.
            init_lr {float} -- Learning rate after warmup.

        Keyword Arguments:
            verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpLearningRateScheduler, self).__init__()
        self.warmup_batches = warmup_batches
        self.init_lr = init_lr
        self.verbose = verbose
        self.batch_count = 0
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.batch_count = self.batch_count + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        if self.batch_count <= self.warmup_batches:
            lr = self.batch_count*self.init_lr/self.warmup_batches
            K.set_value(self.model.optimizer.lr, lr)
            if self.verbose > 0:
                print('\nBatch %05d: WarmUpLearningRateScheduler setting learning '
                      'rate to %s.' % (self.batch_count + 1, lr))
warm_up_lr = WarmUpLearningRateScheduler(400, init_lr=0.005)
from keras.models import Sequential
from keras.callbacks.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.layers import BatchNormalization
#fnl=pd.concat([fnl_trn,fnl_tst],0).select_dtypes(exclude='uint8')
batch_size=1024
checkpoint_path = "training_1.hdf5" 
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback =ModelCheckpoint(checkpoint_path, 
monitor='loss',save_best_only=True,save_weights_only=True,verbose=1)
len_input_columns=fnl.shape[1]
model_dae = Sequential()
model_dae.add(Dense(units=len_input_columns,input_shape=(len_input_columns,), activation='relu', dtype='float32', name='Input'))
model_dae.add(Dense(units=int(512), activation='relu', dtype='float32',name='Hidden_1'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(256), activation='relu', dtype='float32',name='Hidden_2'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(512), activation='relu', dtype='float32',name='Hidden_3'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=len_input_columns, activation='linear', dtype='float32', name='Output'))
model_dae.compile(loss='mean_squared_error',optimizer='adam')
hist=model_dae.fit_generator(DAESequence(fnl, batch_size=2048),steps_per_epoch=int(ceil(fnl.shape[0]/batch_size)),epochs=10,workers=2,callbacks=[warm_up_lr])

Using TensorFlow backend.


Epoch 1/10
1072/1072 [==============================] - 32s 30ms/step - loss: 0.0312
Epoch 2/10
1072/1072 [==============================] - 30s 28ms/step - loss: 0.0074
Epoch 3/10
1072/1072 [==============================] - 30s 28ms/step - loss: 0.0059
Epoch 4/10
1072/1072 [==============================] - 30s 28ms/step - loss: 0.0051
Epoch 5/10
1072/1072 [==============================] - 30s 28ms/step - loss: 0.0046
Epoch 6/10
1072/1072 [==============================] - 30s 28ms/step - loss: 0.0045
Epoch 7/10
1072/1072 [==============================] - 30s 28ms/step - loss: 0.0044
Epoch 8/10
1072/1072 [==============================] - 30s 28ms/step - loss: 0.0042
Epoch 9/10
1072/1072 [==============================] - 30s 28ms/step - loss: 0.0042
Epoch 10/10
1072/1072 [==============================] - 30s 28ms/step - loss: 0.0042


In [0]:
path = F"/content/gdrive/My Drive/autoenc_final.hdf5" 
model_dae.save_weights(path)

In [16]:

from tqdm import tqdm
your_new_df=pd.DataFrame()
for i in tqdm([3]):
    intermediate_layer_model = Model(inputs=model_dae.input, outputs=model_dae.layers[i].output)
    z = pd.DataFrame(intermediate_layer_model.predict(fnl))
    columns_names = ['Hidden_'+str(i)+'_'+str(l) for l in range(0, z.shape[1])]
    z.columns=columns_names
    your_new_df=pd.concat([your_new_df,z],1)

100%|██████████| 1/1 [00:43<00:00, 43.57s/it]


In [17]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        df[col]=df[col].fillna(-1)
        df[col]=df[col].replace([np.inf,-np.inf],-1)
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
your_new_df=reduce_mem_usage(your_new_df)

Memory usage of dataframe is 1071.51 MB
Memory usage after optimization is: 535.76 MB
Decreased by 50.0%


In [0]:
trn=pd.read_csv('train_transaction.csv.zip',usecols=['isFraud'])
your_new_df=your_new_df.loc[:trn.shape[0]]
your_new_df['isFraud']=trn['isFraud']
fnl=pd.DataFrame()
cols=[]
val=[]

In [0]:
your_new_df=your_new_df.dropna()
your_new_df=your_new_df.replace([np.inf,-np.inf],0)

In [0]:
ls=list(your_new_df)
ls.remove('isFraud')

In [24]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
trn=[]
for col in tqdm(ls):
    for i in range(50000,590540,40000):
        fit=your_new_df.loc[0:i]
        pdct=your_new_df.loc[i:i+40000]
        mod=lgb.LGBMClassifier(max_bins= 37,max_depth= 40,min_data_in_leaf= 110,min_gain_to_split= 1,num_leaves=48)
        mod.fit(fit[col].values.reshape(-1,1),fit['isFraud'])
        val.append(roc_auc_score(pdct['isFraud'],mod.predict_proba(pdct[col].values.reshape(-1,1))[:,1]))
        trn.append(roc_auc_score(fit['isFraud'],mod.predict_proba(fit[col].values.reshape(-1,1))[:,1]))
        cols.append(col)


100%|██████████| 256/256 [46:52<00:00, 10.99s/it]


In [0]:
fnl['cols']=cols
fnl['validation']=val
fnl['train']=trn
fnl['validation']=fnl.groupby(['cols'])['validation'].transform('mean')
fnl['train']=fnl.groupby(['cols'])['train'].transform('mean')
fnl=fnl[['validation','train','cols']]
fnl=fnl.drop_duplicates(keep='last')

In [26]:
fnl=fnl.sort_values(by='validation')
fnl.tail(100)['cols'].values

array(['Hidden_3_94', 'Hidden_3_114', 'Hidden_3_238', 'Hidden_3_153',
       'Hidden_3_143', 'Hidden_3_106', 'Hidden_3_196', 'Hidden_3_253',
       'Hidden_3_250', 'Hidden_3_78', 'Hidden_3_89', 'Hidden_3_29',
       'Hidden_3_175', 'Hidden_3_37', 'Hidden_3_5', 'Hidden_3_123',
       'Hidden_3_148', 'Hidden_3_15', 'Hidden_3_30', 'Hidden_3_28',
       'Hidden_3_13', 'Hidden_3_124', 'Hidden_3_97', 'Hidden_3_245',
       'Hidden_3_160', 'Hidden_3_77', 'Hidden_3_22', 'Hidden_3_136',
       'Hidden_3_182', 'Hidden_3_86', 'Hidden_3_43', 'Hidden_3_54',
       'Hidden_3_18', 'Hidden_3_90', 'Hidden_3_163', 'Hidden_3_183',
       'Hidden_3_108', 'Hidden_3_192', 'Hidden_3_79', 'Hidden_3_241',
       'Hidden_3_165', 'Hidden_3_142', 'Hidden_3_50', 'Hidden_3_101',
       'Hidden_3_236', 'Hidden_3_209', 'Hidden_3_206', 'Hidden_3_88',
       'Hidden_3_115', 'Hidden_3_151', 'Hidden_3_12', 'Hidden_3_130',
       'Hidden_3_120', 'Hidden_3_159', 'Hidden_3_217', 'Hidden_3_20',
       'Hidden_3_73', 'Hidden_

In [0]:
path = F"/content/gdrive/My Drive/extra__1.pkl" 
your_new_df.to_pickle(path)